In [17]:
# !pip install transformers

In [18]:
# !pip install NERDA

In [1]:
import numpy as np
import pandas as pd
import json
import time
import re
from NERDA.models import NERDA
import torch
import pickle
import sklearn
import random
from sklearn.preprocessing import MultiLabelBinarizer

/home/hl973/.local/lib/python3.7/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [2]:
# from NERDA.datasets import get_conll_data, download_conll_data 
# download_conll_data()
# training = get_conll_data('train')
# validation = get_conll_data('valid')

Reading https://data.deepai.org/conll2003.zip


In [3]:
# print(training.keys())
# print(training['sentences'])
# print(training['tags'])
# print(np.array(training['sentences']).shape)
# print(len((training['tags'])))

In [4]:
import transformers
from transformers import BertConfig, BertTokenizer, BertModel

model_dir = "bert-base-chinese"

config = BertConfig.from_pretrained(model_dir)
tokenizer = BertTokenizer.from_pretrained(model_dir)
model = BertModel.from_pretrained(model_dir)

In [5]:
print(type(model))

<class 'transformers.modeling_bert.BertModel'>


In [6]:
# data in result.json is the labeled ming data
# with open('/home/aistudio/result.json') as jf:
#     ming=json.load(jf)
with open('result.json') as jf:
    ming=json.load(jf)

#unpack .json file, wash the data and transfer them into ndarray
# x_data, y_data corresponds to texts and tags
def dataGen(ming):
    person_ids=np.array(list(ming.keys()))
    person_ids.sort()

    #idNum=len(person_ids)

    x_data=[]
    y_data=[]

    indexer=0
    for person_id in person_ids:
        
        char_tag=ming[person_id]['char_tag']
        x_data.append([])
        y_data.append([])
        omit_len=len(person_id)
        for i in range(omit_len+1,len(char_tag)):
            x_data[indexer].append(char_tag[i][0])
            y_data[indexer].append(char_tag[i][1])
         
        indexer=indexer+1

    for i in range(0,len(y_data)):
        for j in range(0,len(y_data[i])):
            old_text=y_data[i][j]
            #convert labels like 'B_date_reign' into 'B-date-reign'
            new_text=old_text.replace("_","-")
            y_data[i][j]=new_text


    return x_data,y_data,person_ids


#construct train,validate and test set
#train_set_rate indicates the proportion of trainning data
#validate_set_rate indicates the proportion of validation data
def splitTrain(x_data,y_data,person_ids,train_set_rate,validate_set_rate):
    x_data=np.array(x_data)
    y_data=np.array(y_data)
    
    temp=np.array([x_data,y_data])
    temp=temp.T
    
    ming_data=pd.DataFrame(temp,index=person_ids,columns=['text_a','label'])

    #To save data as .tsv files, we must separate the characters using \002 marker
    for i in range(0,len(ming_data['text_a'])):
        #print(i)
        ming_data['text_a'][i]='\002'.join(ming_data['text_a'][i])
        ming_data['label'][i]='\002'.join(ming_data['label'][i])
        ##ming_data['text_a'][i]+='\002'
        ##ming_data['label'][i]+='\002'
        #ming_data['text_a'][i]=str(ming_data['text_a'][i])
        #ming_data['label'][i]=str(ming_data['label'][i])
        
        

    np.random.seed(int(time.time()))
    ming_data=ming_data.sample(frac=1.0)
    
    idNum=len(person_ids)
    train_size=int(np.floor(idNum*train_set_rate))
    validate_size=int(np.floor(idNum*validate_set_rate))

    train_set=ming_data[0:train_size]

    validate_set=ming_data[train_size:train_size+validate_size]
    test_set=ming_data[train_size+validate_size:idNum]
    return train_set,validate_set,test_set,ming_data


#Build the first model: 50% of ming as train and 20% of ming as test
x_data,y_data,person_ids=dataGen(ming)
t,v,testing,ming_data=splitTrain(x_data,y_data,person_ids,0.7,0.15)

#generate x,y of train,validate and test
x_train=np.array(t['text_a'])
y_train=np.array(t['label'])
x_validate=np.array(v['text_a'])
y_validate=np.array(v['label'])

x_test=np.array(testing['text_a'])
y_test=np.array(testing['label'])

# We can save them as .csv files if we want
t.to_csv('train.csv')
v.to_csv('validate.csv')

# Acquire the length of training set
train_len=[]
for i in range(0,len(x_train)):
    train_len.append(len(x_train[i]))

max_len=max(train_len)

# The tag list, which specifies how many tags are of interest.
tag_list = tag_list = ["O",
        "B_date_reign", "I_date_reign",
        "B_date_year", "I_date_year",
        "B_office_voa", "I_office_voa",
        "B_office_title", "I_office_title",
        "B_place_placename", "I_place_placename"]

In [7]:
MAX_LENGTH = 512

In [8]:
sets = [x_train, x_validate, y_train, y_validate, x_test, y_test]
final_sets = [] 
for group in sets:
    data_lst = []
    for sent_or_label in group:
        tokenized = sent_or_label.split('\x02')
        if len(tokenized) > MAX_LENGTH:
            tokenized = sent_or_label.split('\x02')[:MAX_LENGTH]
            data_lst.append(tokenized)
        elif (len(tokenized) > 1 or tokenized[0] != ''):
            data_lst.append(tokenized)
        else:
          # Do not include
          pass
    final_sets.append(data_lst)

train_sent, valid_sent, train_labels, valid_labels, test_sent, test_labels = final_sets[0], final_sets[1], final_sets[2], final_sets[3], final_sets[4], final_sets[5]


In [9]:
print(len(x_test))
print(len(x_validate))
print(len(x_train))

1330
1328
6200


In [10]:
training_dict = {'sentences': train_sent, 'tags': train_labels}
validation_dict = {'sentences': valid_sent, 'tags': valid_labels}
test_dict = {'sentences': test_sent, 'tags':test_labels}

In [11]:
len(training_dict['sentences'])

6199

In [9]:
# import pickle

In [46]:
# pickle.dump(test_dict, open('test_dict.pkl', 'wb'))

In [12]:
print(np.array(train_sent).shape)
print((train_sent[0]))
print(len(train_labels))
print(len(train_labels[0]))
print(len(valid_sent))

(6199,)
['字', '宗', '儒', '，', '蒲', '圻', '人', '。', '天', '眷', '間', '擧', '人', '，', '授', '中', '書', '檢', '正', '官', '，', '持', '躬', '端', '恪', '，', '禮', '士', '恤', '民', '。', '陞', '弘', '文', '院', '校', '理', '，', '承', '辦', '釆', '木', '之', '役', '，', '親', '歷', '知', '城', '，', '調', '宮', '苑', '司', '。']
6199
55
1327


In [13]:
# hyperparameters for network
dropout = 0.1
# hyperparameters for training
training_hyperparameters = {
'epochs' : 4,
'warmup_steps' : 200,
'train_batch_size': 13,
'learning_rate': 0.0001
}

In [14]:
model = NERDA(
dataset_training = training_dict,
dataset_validation = validation_dict,
tag_scheme = [
        "B-date-reign", "I-date-reign",
        "B-date-year", "I-date-year",
        "B-office-voa", "I-office-voa",
        "B-office-title", "I-office-title",
        "B-place-placename", "I-place-placename"], 
tag_outside = 'O',
transformer = model_dir,
dropout = dropout,
hyperparameters = training_hyperparameters,
max_len = 512
)

Device automatically set to: cpu


In [48]:
model.train()

  0%|          | 0/511 [00:00<?, ?it/s]


 Epoch 1 / 4


 12%|█▏        | 60/511 [29:27<3:41:29, 29.47s/it]


KeyboardInterrupt: 

In [15]:
# path = ''

In [16]:
# # Part of Task1: save the data into tsv
# # t-> train, v-> validate, testing -> test
# # ming_data -> whole dataset, also saved in case it might be of use
# t.to_csv((path + 'train.tsv'),sep='\t',columns=['text_a','label'],encoding='utf_8_sig',index=None)
# v.to_csv((path + 'validate.tsv'),sep='\t',columns=['text_a','label'],encoding='utf_8_sig',index=None)
# testing.to_csv((path + 'testing.tsv'),sep='\t',columns=['text_a','label'],encoding='utf_8_sig',index=None)
# ming_data.to_csv((path + 'dataset.tsv'),sep='\t',columns=['text_a','label'],encoding='utf_8_sig',index=None)

# # predict_data is the Jin biographical data without tags
# predict_data=pd.read_table('test_data.txt')

# # 'content_without_name' consists of characters in the texts
# text=predict_data['content_without_name']

# # Also save it as .tsv files in case
# text.to_csv((path + 'predict.tsv'),encoding='utf_8_sig')

In [15]:
model.load_network_from_file(model_path='NERDA_model.bin')

'Weights for network loaded from NERDA_model.bin'

In [16]:
test_sent[0]

['字',
 '鳴',
 '玉',
 '，',
 '耒',
 '陽',
 '人',
 '。',
 '諸',
 '生',
 '，',
 '天',
 '會',
 '十',
 '六',
 '年',
 '闖',
 '賊',
 '犯',
 '耒',
 '陽',
 '，',
 '如',
 '珂',
 '拒',
 '戰',
 '死',
 '之',
 '。']

In [21]:
model.evaluate_performance(validation_dict)

Traceback (most recent call last):
  File "/share/apps/anaconda3/2020.02/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/share/apps/anaconda3/2020.02/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/share/apps/anaconda3/2020.02/lib/python3.7/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/share/apps/anaconda3/2020.02/lib/python3.7/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe


KeyboardInterrupt: 

In [29]:
sentence = ['河 內 人 天 會 十 四 年 參 與 修 北 村 湯 王 廟'.split()]

In [30]:
sentence

[['河',
  '內',
  '人',
  '天',
  '會',
  '十',
  '四',
  '年',
  '參',
  '與',
  '修',
  '北',
  '村',
  '湯',
  '王',
  '廟']]

In [28]:
(x_data[0])

['字',
 '一',
 '菴',
 '，',
 '號',
 '退',
 '翁',
 '，',
 '上',
 '虞',
 '孫',
 '氏',
 '子',
 '。',
 '年',
 '十',
 '三',
 '為',
 '僧',
 '，',
 '居',
 '杭',
 '州',
 '上',
 '天',
 '竺',
 '寺',
 '。',
 '興',
 '定',
 '中',
 '奉',
 '命',
 '輯',
 '禪',
 '宗',
 '語',
 '錄',
 '，',
 '後',
 '又',
 '輯',
 '法',
 '華',
 '科',
 '注',
 '，',
 '大',
 '明',
 '法',
 '數',
 '等',
 '書',
 '，',
 '屢',
 '授',
 '鼻',
 '骨',
 '德',
 '部',
 '都',
 '監',
 '，',
 '年',
 '七',
 '十',
 '四',
 '卒',
 '。']

In [ ]:
group_1_test = []

In [70]:
# assert len(x_data) == len(y_data)
# indices = [i for i in range(len(x_data))]
# random.shuffle(indices)
# y_true = [y_data[i] for i in range(500)]
# x_subset = [x_data[i] for i in range(500)]
# y_pred = model.predict(x_subset)
y_true = test_dict['tags'][:10]
y_pred = model.predict(test_dict['sentences'][:10])

In [71]:
mlb = MultiLabelBinarizer()
mlb.fit(y_true)
y_true_binary = mlb.transform(y_true)
y_pred_binary = mlb.transform(y_pred)

In [72]:
sklearn.metrics.precision_score(y_true_binary, y_pred_binary, average='weighted')

1.0

In [73]:
sklearn.metrics.recall_score(y_true_binary, y_pred_binary, average='weighted' )

1.0